In [1]:
# Cell 0
import pandas as pd
import numpy as np
from pathlib import Path
import textwrap

# === Đường dẫn chỉnh lại theo máy của bạn ===
ROOT = Path("/home/kylh/phd/tw_fin_rl/work/data/gdelt_0024_hardfix_split/normalized_unique")
ARTICLES_PATH = ROOT / "articles.parquet"
REL_PATH      = ROOT / "article_token_map.parquet"

# Load
articles = pd.read_parquet(ARTICLES_PATH)
rel = pd.read_parquet(REL_PATH)

# Chuẩn kiểu thời gian
for c in ["first_seen","last_seen"]:
    if c in rel.columns:
        rel[c] = pd.to_datetime(rel[c], utc=True, errors="coerce")

for c in ["first_seen_any","last_seen_any","seendate"]:
    if c in articles.columns:
        articles[c] = pd.to_datetime(articles[c], utc=True, errors="coerce")

# Token list (toàn bộ)
ALL_TOKENS = sorted(rel["symbol_req"].dropna().unique().tolist())
ALL_TOKENS[:8], len(ALL_TOKENS)


(['BNB', 'BTC', 'ETH', 'SOL'], 4)

In [2]:
# Cell 1
def expand_days(rel_df: pd.DataFrame) -> pd.DataFrame:
    """
    Mở rộng days_list -> mỗi dòng là 1 ngày cụ thể cho 1 (url, token).
    Output: columns = [url_canon, symbol_req, day (datetime), first_seen, last_seen, n_hits]
    """
    tmp = rel_df.copy()
    # đảm bảo days_list là list
    tmp["days_list"] = tmp["days_list"].apply(lambda x: x if isinstance(x, (list, tuple, np.ndarray)) else [])
    tmp = tmp.explode("days_list", ignore_index=True)
    tmp = tmp.rename(columns={"days_list": "day"})
    tmp["day"] = pd.to_datetime(tmp["day"], format="%Y-%m-%d", errors="coerce")
    return tmp

def period_counts(df_days: pd.DataFrame, token: str):
    """
    Tạo phân bố theo ngày/tuần/tháng/năm cho 1 token.
    day_count: số URL unique theo ngày
    week_count: số URL unique theo tuần (ISO-week)
    month_count, year_count tương tự
    """
    tdf = df_days[df_days["symbol_req"] == token].copy()
    if tdf.empty:
        return (pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

    # Đếm URL unique theo period
    day_count = (tdf.dropna(subset=["day"])
                    .groupby("day")["url_canon"]
                    .nunique()
                    .rename("n_urls")
                    .reset_index())

    # Tuần: dùng Mon-Sun ISO tuần (to_period("W-MON") để tuần bắt đầu T2)
    tdf["week"] = tdf["day"].dt.to_period("W-MON")  # tuần bắt đầu Monday
    week_count = (tdf.dropna(subset=["week"])
                    .groupby("week")["url_canon"]
                    .nunique()
                    .rename("n_urls")
                    .reset_index())
    week_count["week_start"] = week_count["week"].dt.start_time
    week_count["week_end"] = week_count["week"].dt.end_time

    # Tháng
    tdf["month"] = tdf["day"].dt.to_period("M")
    month_count = (tdf.dropna(subset=["month"])
                     .groupby("month")["url_canon"]
                     .nunique()
                     .rename("n_urls")
                     .reset_index())
    month_count["month_start"] = month_count["month"].dt.start_time
    month_count["month_end"] = month_count["month"].dt.end_time

    # Năm
    tdf["year"] = tdf["day"].dt.to_period("Y")
    year_count = (tdf.dropna(subset=["year"])
                    .groupby("year")["url_canon"]
                    .nunique()
                    .rename("n_urls")
                    .reset_index())
    year_count["year_start"] = year_count["year"].dt.start_time
    year_count["year_end"] = year_count["year"].dt.end_time

    return day_count, week_count, month_count, year_count

def token_basic_stats(rel_df: pd.DataFrame, token: str) -> dict:
    """
    In stats lõi cho 1 token: số URL unique, khoảng thời gian, tổng số hit (n_hits), top domain…
    """
    t = rel_df[rel_df["symbol_req"] == token].copy()
    if t.empty:
        return {"token": token, "n_urls": 0}

    n_urls = t["url_canon"].nunique()
    first_seen = t["first_seen"].min()
    last_seen  = t["last_seen"].max()
    total_hits = t["n_hits"].sum()
    median_hits = t["n_hits"].median()

    # Top 10 domain theo số URL unique
    # Join qua articles để lấy domain/title nếu cần
    dom = (t[["url_canon"]].drop_duplicates()
             .merge(articles[["url_canon","domain"]], on="url_canon", how="left"))
    top_domains = (dom.groupby("domain")["url_canon"].nunique()
                     .sort_values(ascending=False)
                     .head(10))

    return {
        "token": token,
        "n_urls": int(n_urls),
        "first_seen": first_seen,
        "last_seen": last_seen,
        "total_hits": int(total_hits),
        "median_hits_per_url": float(median_hits) if pd.notnull(median_hits) else None,
        "top_domains": top_domains
    }

def pretty_print_token_stats(stats: dict, max_domains: int = 10):
    print(f"=== {stats['token']} ===")
    print(f"- Unique URLs           : {stats['n_urls']:,}")
    print(f"- First seen (UTC)      : {stats.get('first_seen')}")
    print(f"- Last seen  (UTC)      : {stats.get('last_seen')}")
    print(f"- Total hits (sum)      : {stats.get('total_hits'):,}")
    print(f"- Median hits per URL   : {stats.get('median_hits_per_url')}")
    if "top_domains" in stats and isinstance(stats["top_domains"], pd.Series):
        print("- Top domains by unique URLs:")
        for dom, cnt in stats["top_domains"].head(max_domains).items():
            print(f"    {dom or '<NA>'}: {cnt}")
    print()


In [3]:
# Cell 2
rel_days = expand_days(rel)

# Ví dụ: chọn các token chính (theo bạn muốn)
TOKENS = ["BTC", "ETH", "BNB", "SOL"]

all_dists = {}
for tok in TOKENS:
    day_c, week_c, month_c, year_c = period_counts(rel_days, tok)
    all_dists[tok] = {
        "day": day_c,
        "week": week_c,
        "month": month_c,
        "year": year_c,
    }

# Stats cơ bản cho từng token
for tok in TOKENS:
    st = token_basic_stats(rel, tok)
    pretty_print_token_stats(st)


=== BTC ===
- Unique URLs           : 105,772
- First seen (UTC)      : 2020-08-11 00:15:00+00:00
- Last seen  (UTC)      : 2025-09-03 23:30:00+00:00
- Total hits (sum)      : 114,830
- Median hits per URL   : 1.0
- Top domains by unique URLs:
    biztoc.com: 11369
    forextv.com: 5804
    cointelegraph.com: 3367
    newsbtc.com: 3262
    benzinga.com: 2078
    fxstreet.com: 1961
    forbes.com: 1776
    insidebitcoins.com: 1705
    coindesk.com: 1700
    coinspeaker.com: 1625

=== ETH ===
- Unique URLs           : 25,927
- First seen (UTC)      : 2020-08-11 09:30:00+00:00
- Last seen  (UTC)      : 2025-09-03 19:15:00+00:00
- Total hits (sum)      : 29,695
- Median hits per URL   : 1.0
- Top domains by unique URLs:
    biztoc.com: 2618
    fxstreet.com: 1305
    newsbtc.com: 1039
    coinspeaker.com: 761
    benzinga.com: 706
    cointelegraph.com: 688
    fool.com: 509
    economictimes.indiatimes.com: 472
    invezz.com: 415
    insidebitcoins.com: 382

=== BNB ===
- Unique URLs    

In [4]:
# Cell 4
def show_distribution_summary(dists: dict, token: str, head_n: int = 5):
    print(f"### Distribution for {token}")
    for k in ["day","week","month","year"]:
        dfk = dists[token][k]
        if dfk.empty:
            print(f"- {k}: <empty>")
            continue
        print(f"- {k}: {len(dfk)} rows | sample:")
        print(dfk.head(head_n))
    print()

for tok in TOKENS:
    show_distribution_summary(all_dists, tok, head_n=5)


### Distribution for BTC
- day: 1816 rows | sample:
         day  n_urls
0 2020-08-11      51
1 2020-08-12      58
2 2020-08-13      48
3 2020-08-14      53
4 2020-08-15      26
- week: 263 rows | sample:
                    week  n_urls week_start                      week_end
0  2020-08-11/2020-08-17     322 2020-08-11 2020-08-17 23:59:59.999999999
1  2020-08-18/2020-08-24     314 2020-08-18 2020-08-24 23:59:59.999999999
2  2020-08-25/2020-08-31     309 2020-08-25 2020-08-31 23:59:59.999999999
3  2020-09-01/2020-09-07     305 2020-09-01 2020-09-07 23:59:59.999999999
4  2020-09-08/2020-09-14     420 2020-09-08 2020-09-14 23:59:59.999999999
- month: 62 rows | sample:
     month  n_urls month_start                     month_end
0  2020-08     945  2020-08-01 2020-08-31 23:59:59.999999999
1  2020-09    1305  2020-09-01 2020-09-30 23:59:59.999999999
2  2020-10     967  2020-10-01 2020-10-31 23:59:59.999999999
3  2020-11    1596  2020-11-01 2020-11-30 23:59:59.999999999
4  2020-12    2299 

In [5]:
import pandas as pd
import numpy as np

def daily_spikes(rel_days: pd.DataFrame, token: str, win=28, z_th=2.5):
    """
    Tìm ngày 'bùng nổ' news cho token dựa trên z-score rolling.
    Trả về bảng: day, n_urls, mean, std, z, is_spike
    """
    t = rel_days[rel_days["symbol_req"] == token].dropna(subset=["day"]).copy()
    day_count = (t.groupby("day")["url_canon"].nunique()
                   .rename("n_urls").to_frame().sort_index())
    # rolling stats
    roll_mean = day_count["n_urls"].rolling(win, min_periods=max(7, win//4)).mean()
    roll_std  = day_count["n_urls"].rolling(win, min_periods=max(7, win//4)).std(ddof=0)
    z = (day_count["n_urls"] - roll_mean) / roll_std.replace(0, np.nan)
    out = day_count.assign(mean=roll_mean, std=roll_std, z=z, is_spike=z >= z_th)
    return out.reset_index()

# ví dụ:
btc_spikes = daily_spikes(rel_days, "BTC", win=28, z_th=2.5)
btc_spikes[btc_spikes["is_spike"]].tail(10)


,day,n_urls,mean,std,z,is_spike
1177,2023-11-09,133,65.071429,24.922635,2.725577,True
1202,2023-12-04,150,70.857143,30.980079,2.554637,True
1230,2024-01-09,151,76.750000,29.137145,2.548294,True
1280,2024-02-28,152,66.642857,33.861287,2.520789,True
1535,2024-11-10,92,39.321429,16.896858,3.117655,True
1536,2024-11-11,161,44.392857,27.817440,4.191872,True
1560,2024-12-05,168,68.607143,34.767944,2.858750,True
1642,2025-02-25,99,45.928571,18.704332,2.837387,True
1652,2025-03-07,106,50.607143,21.579321,2.566942,True
1727,2025-05-21,134,52.428571,26.208544,3.112398,True


In [7]:
# Fix monthly_norm: always use a single datetime column `ts`
import pandas as pd
from pandas.api.types import is_period_dtype

def monthly_norm(all_dists: dict, tokens=("BTC","ETH","BNB","SOL")) -> pd.DataFrame:
    frames = []
    for tok in tokens:
        dfm = all_dists[tok]["month"].copy()
        if dfm.empty:
            continue

        # Build a single datetime column `ts` for the month point (start of month)
        if "month_start" in dfm.columns:
            ts = pd.to_datetime(dfm["month_start"], utc=True, errors="coerce")
        elif "month" in dfm.columns and is_period_dtype(dfm["month"]):
            ts = dfm["month"].dt.start_time
        else:
            ts = pd.to_datetime(dfm["month"], utc=True, errors="coerce")

        frames.append(pd.DataFrame({
            "ts": ts,
            "n_urls": pd.to_numeric(dfm["n_urls"], errors="coerce"),
            "token": tok
        }))

    if not frames:
        return pd.DataFrame()

    long = pd.concat(frames, ignore_index=True).dropna(subset=["ts"])
    # Aggregate in case duplicates exist for same (ts, token)
    wide = (long.pivot_table(index="ts", columns="token", values="n_urls", aggfunc="sum")
                 .sort_index())

    # z-score by column (guard against zero std)
    std = wide.std(ddof=0).replace(0, pd.NA)
    wide_z = (wide - wide.mean()) / std
    return wide_z.reset_index()  # ts + tokens


In [8]:
monthly_z = monthly_norm(all_dists, tokens=["BTC","ETH","BNB","SOL"])
monthly_z.tail()


token,ts,BNB,BTC,ETH,SOL
57,2025-05-01 00:00:00+00:00,-0.614206,0.323293,-0.952247,-0.362839
58,2025-06-01 00:00:00+00:00,-0.647575,-1.667718,-1.290379,-1.054830
59,2025-07-01 00:00:00+00:00,0.153269,0.790490,-0.531549,-0.118607
60,2025-08-01 00:00:00+00:00,-0.547469,0.041397,-0.547276,-0.240723
61,2025-09-01 00:00:00+00:00,-0.847785,-3.041712,-1.577397,-1.095536
